In [240]:
import pandas as pd
from pandas import option_context
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import MWETokenizer
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer 

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

from collections import Counter

from textblob import TextBlob

## This is the main notebook for topic modeling and sentiment analysis (version 2)

In [241]:
df = pd.read_csv('train.csv')
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
df.shape

(99296, 24)

In [242]:
df_analysis = df[['username','tweet']].copy()
df_analysis.dropna(inplace=True)
df_analysis.reset_index(drop=True,inplace=True)

### Count words

In [243]:
stop = stopwords.words('english')
stop.extend(['hi', 'hey', 'hello','ha', 'followed','wa','dm','dont','cant','wont','get','still','like','need',
            'someone','people','im','ive','month','week','day','could','give','want','please','pls','since','one',
             'back','thanks','thank','take','doesnt','does', 'might', 'must','lock','access','sent','closed','use',
            'u','contact','new','email','time','message','tried','answer','waiting','issue','reason','going'])

In [244]:
# # Extend for most common English adverbs

stop.extend(['up','so','out','just','now','how','then','more','also','here',
            'well','only','very','even','back','there','down','still','in',
            'as','to','when','never','really','most','on','why','about','over',
            'again','where','right','off','always','today','all','far','long',
            'away','yet','often','ever','however','almost','later','much',
            'once','least','ago','together','around','already','enough','both',
            'maybe','actually','probably','home','of course','perhaps','little',
            'else','sometimes','finally','less','better','early','especially',
            'either','quite','simply','nearly','soon','certainly','quickly',
            'no','recently','before','usually','thus','exactly','hard',
            'particularly','pretty','forward','ok','okay','clearly','indeed',
            'rather','that','tonight','close','suddenly','best','instead',
            'ahead','fast','alone','eventually','directly'])

# # Extend for most common irregular verbs (except pay,lose,send,buy,spend)

stop.extend(['say','make','go','take','come','see','know','get','got','give',
            'find','think','tell','become','show','leave','feel','put','bring',
            'begin','keep','hold','write','stand','hear','let','mean','set','meet',
            'run','sit','speak','lie','lead','read','grow','fall',
            'build','understand','draw','break','cut','rise','drive','wear',
            'choose'])

# ## Extend for prepositions
stop.extend(['without','among'])

#### Compounds

In [245]:
create_compounds = (lambda x: 
                 x.replace('debit card', 'debitcard')
                 .replace('rainbow card', 'debitcard')
                 .replace('bank card', 'debitcard')
                 .replace('revolut card', 'debitcard')
                 .replace('revcards', 'debitcard')
                 .replace('credit card', 'creditcard')
                 .replace('junior card', 'juniorcard')
                 .replace('revolut junior', 'juniorcard')
                 .replace('business account', 'budinessaccount')
                 .replace('savings account', 'savingsaccount')
                 .replace('bank account', 'bankaccount')
                 .replace('premium account', 'premiumaccount')
                 .replace('premium user', 'premiumaccount')
                 .replace('premium plan', 'premiumaccount')
                 .replace('metal account', 'metalaccount')
                 .replace('metal card', 'metalaccount')
                 .replace('metal customers', 'metalaccount')
                 .replace('metal user', 'metalaccount')
                 .replace('business bank', 'businessbank')
                 .replace('virtual card', 'virtualcard')
                 .replace('revolut business', 'revolutbusiness')
                 .replace('google pay', 'googlepay')
                 .replace('apple pay', 'applepay')
                 .replace('applepayment', 'applepay')
                 .replace('samsung pay', 'samsungpay')
                 .replace('cryptocurrency', 'crypto')
                 .replace('cryptos', 'crypto')
                 .replace('cryptoasset', 'crypto')
                 .replace('doge coin', 'crypto')
                 .replace(' doge ', ' crypto ')
                 .replace('customer service', 'customerservice')
                 .replace('supoort', 'support')
                 .replace('customer support', 'customerservice')
                 .replace(' cs ', ' customerservice ')
                 .replace('phone number', 'phonenumber')
                 .replace('social media', 'socialmedia')
                 .replace('app chat', 'appchat')
                 .replace('challenger bank', 'fintech')
                 .replace('challengerbank', 'fintech')
                 .replace('neobanking', 'fintech')
                 .replace('neobanks', 'fintech')
                 .replace('neobank', 'fintech')
                 .replace('transferthis', 'transfer')
                 .replace('application', 'app')
                 .replace('locked', 'lock')
                 .replace('unlock', 'lock')
                 .replace('block', 'lock')
                 .replace('dark mode', 'darkmode')
                 .replace('wealth management', 'wealthmanagement'))


df_analysis['tweet_compound'] = df_analysis.tweet.map(create_compounds)

#### Count

In [246]:
counter = Counter()
lemma = nltk.stem.WordNetLemmatizer()

for tweet in df_analysis['tweet_compound']:
    tweet = lemma.lemmatize(tweet)
    tweet = TextBlob(tweet).words  # tokenize words
    tweet = [w for w in tweet if w not in stop]

    counter += Counter(tweet)

In [247]:
len(counter)

32754

In [248]:
for phrase, count in counter.most_common(10):
    print('%20s %i' % ("".join(phrase), count))

             account 29068
             revolut 17033
                 app 16213
               money 15544
                help 12151
                chat 7750
                card 6317
             support 5496
                days 5380
               agent 4469


### Topic modeling

In [249]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]
    
vectorizer = CountVectorizer(tokenizer = LemmaTokenizer(),
                              stop_words = stop,
                              min_df=2,
                              max_df = 0.9)

doc_word = vectorizer.fit_transform(df_analysis.tweet_compound)
doc_word.shape

/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'course', 'doe', 'le', "n't", 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


(99296, 13152)

In [250]:
nmf_model = NMF(5)
doc_topic = nmf_model.fit_transform(doc_word)
doc_topic.shape

(99296, 5)

In [251]:
topic_word = nmf_model.components_
topic_word.shape

(5, 13152)

In [252]:
words = vectorizer.get_feature_names()
t = nmf_model.components_.argsort(axis=1)[:,-1:-7:-1]
topic_words = [[words[e] for e in l] for l in t]
topic_words

[['account', 'fund', 'document', 'open', 'information', 'customerservice'],
 ['app', 'phone', 'card', 'support', 'log', 'work'],
 ['revolut', 'bank', 'crypto', 'customer', 'fintech', 'child'],
 ['money', 'bank', 'transfer', 'send', 'card', 'pay'],
 ['help', 'chat', 'agent', 'live', 'support', 'card']]

- Component 0 (topic 1) seems to be about account queries
- Component 1 (topic 2) seems to be about app-related queries
- Component 2 (topic 3) seems to be about fintech innovations
- Component 3 (topic 4) is about transfers and not being able to access accounts / money
- Component 4 (topic 5) is about general requests for support

In [253]:
df_analysis['topic'] = doc_topic.argmax(axis=1)

In [254]:
with option_context('display.max_colwidth', 600):
    display(df_analysis.sample(5))

,username,tweet,tweet_compound,topic
14680,tweetytweet2019,im really sad about it what if i choose to close down my accounts at this stage what happened to my balances,im really sad about it what if i choose to close down my accounts at this stage what happened to my balances,0
93057,starlingbank,hi shell were definitely here to stay if youd like to know anything more just give us a shout,hi shell were definitely here to stay if youd like to know anything more just give us a shout,2
76268,skamelone,hi my ios application is stuck at the identity check failed screen and theres no way to exit it i tried to send all the information at least x good quality not blurry nothing seems to work and now i cant even use the money i put in my account help,hi my ios app is stuck at the identity check failed screen and theres no way to exit it i tried to send all the information at least x good quality not blurry nothing seems to work and now i cant even use the money i put in my account help,1
55880,f_fyvie,still not able to transfer to any wallet though,still not able to transfer to any wallet though,3
86000,workingkills,i guess the ui is alsofrozen,i guess the ui is alsofrozen,1


### Sentiment analysis

In [156]:
sid_obj = SentimentIntensityAnalyzer()
sentiment = []
for tweet in df_analysis.tweet_compound:
    sentiment.append(sid_obj.polarity_scores(tweet))

In [157]:
sentiment_df = pd.DataFrame(sentiment)
sentiment_df.head()

,neg,neu,pos,compound
0,0.000,0.722,0.278,0.4019
1,0.329,0.671,0.000,-0.9136
2,0.138,0.862,0.000,-0.7269
3,0.000,0.952,0.048,0.2023
4,0.139,0.696,0.165,-0.1531


In [158]:
merged_df = pd.concat([df_analysis, sentiment_df], axis=1)

In [159]:
full_df = pd.concat([df,merged_df],axis=1)

### Analysis - high level topics

In [160]:
## Add month feature to enable analysis over time
full_df['month'] = pd.DatetimeIndex(full_df['date']).month
full_df['day'] = pd.DatetimeIndex(full_df['date']).day

In [161]:
with option_context('display.max_colwidth', 600):
    display(full_df[(full_df['topic']==2)].sample(2))

,Unnamed: 0,id,conversation_id,created_at,date,time,timezone,user_id,username,tweet,...,username,tweet,tweet_compound,topic,neg,neu,pos,compound,month,day
68188,68188,1259840548543827972,1259840548543827972,2020-05-11 09:39:28 EDT,2020-05-11,09:39:28,-500,2817365839,jarokrolewski,really cool for wallet management,...,jarokrolewski,really cool for wallet management,really cool for wallet management,2,0.0,0.607,0.393,0.3804,5,11
92024,92024,1229375718183915520,1229375718183915520,2020-02-17 07:03:06 EST,2020-02-17,07:03:06,-500,1412497622,clausematch,digital bank has more than tripled its valuation to bn bn after closing the funding round heres how this fast growing fintech ensures compliance,...,clausematch,digital bank has more than tripled its valuation to bn bn after closing the funding round heres how this fast growing fintech ensures compliance,digital bank has more than tripled its valuation to bn bn after closing the funding round heres how this fast growing fintech ensures compliance,2,0.0,0.931,0.069,0.1779,2,17


In [162]:
full_df.groupby(['topic','month'])['compound'].mean()

topic  month
0      1        0.031812
       2        0.021700
       3        0.021529
       4        0.043448
       5        0.041336
       6       -0.010325
       7        0.026898
       8        0.009379
       9        0.041506
       10       0.043509
       11       0.026092
       12       0.047148
1      1        0.066609
       2        0.091436
       3        0.102382
       4        0.057057
       5        0.083351
       6        0.056466
       7        0.087909
       8        0.094616
       9        0.087912
       10       0.104791
       11       0.076863
       12       0.082567
2      1        0.119911
       2        0.197166
       3        0.167375
       4        0.130164
       5        0.110703
       6        0.078551
       7        0.161052
       8        0.112354
       9        0.143191
       10       0.135144
       11       0.043564
       12       0.184340
3      1       -0.048408
       2       -0.012772
       3        0.021191
       4    

In [163]:
full_df.groupby('topic')['topic'].count()

topic
0    15340
1    22547
2    19388
3    14005
4    28016
Name: topic, dtype: int64

In [164]:
full_df.shape

(99296, 34)

In [165]:
## Export to csv for Tableau analysis
#full_df.to_csv('datatableau.csv')

### TFI-DF

In [166]:
# function to display top n terms associated with each topic

def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [167]:
# tuning vectorizer params
tf_idf = TfidfVectorizer(stop_words=stop,
                         tokenizer=word_tokenize,
                         min_df= 2,
                         max_df= 0.9)

# document-term matrix
doc_word2 = tf_idf.fit_transform(df_analysis.tweet_compound)
print(doc_word2.shape)

/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'course', "n't", 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


(99296, 14602)


In [168]:
n = 5 # number of topics

# model selection, fit/trans, and hyperparameter tuning
nmf_model_2 = NMF(n_components =n)

# doc-topic matrix
doc_topic2 = nmf_model_2.fit_transform(doc_word2)

# creating ids for each topic
topic_ids2 = ["topic"+str(val) for val in range(n)]

# topic-term matrix
topic_word2 = pd.DataFrame(nmf_model_2.components_.round(n),
             index = topic_ids2,
             columns = tf_idf.get_feature_names())

# prints top x words in each topic
display_topics(nmf_model_2, 
               tf_idf.get_feature_names(), 
               5) # number of top words/topic


Topic  0
account, lock, access, months, reason

Topic  1
help, account, problem, revolut, trying

Topic  2
app, access, contact, support, new

Topic  3
waiting, chat, agent, live, reply

Topic  4
money, revolut, bank, transfer, send


In [169]:
for tweet in merged_df['tweet_compound']:
    tweet.strip()

In [170]:
tr, test = train_test_split(merged_df,train_size = 0.05,random_state=10)

In [171]:
#tr.to_csv('data_training.csv')

### Intent analysis

In [258]:
df_intent_tr = pd.read_csv('data_training.csv')

In [259]:
df_intent_tr.dropna(inplace=True)
df_intent_tr.reset_index(drop=True,inplace=True)

In [260]:
df_intent_tr.shape

(1207, 10)

In [261]:
df_intent_tr.head(2)

,Unnamed: 0,username,tweet,tweet_compound,intent,topic,neg,neu,pos,compound
0,89389.0,mstothard,can it sustain its evergrowing costs and...,can it sustain its evergrowing costs and...,5.0,2.0,0.0,0.88,0.12,0.743
1,65049.0,saadishtiaq,is premium and metal account subscription bas...,is premium and metalaccount subscription base...,3.0,2.0,0.0,1.00,0.00,0.000


In [262]:
df_intent_tr.groupby('intent').count()

,Unnamed: 0,username,tweet,tweet_compound,topic,neg,neu,pos,compound
intent,,,,,,,,,
0.0,448,448,448,448,448,448,448,448,448
1.0,11,11,11,11,11,11,11,11,11
2.0,166,166,166,166,166,166,166,166,166
3.0,450,450,450,450,450,450,450,450,450
4.0,48,48,48,48,48,48,48,48,48
5.0,5,5,5,5,5,5,5,5,5
6.0,46,46,46,46,46,46,46,46,46
9.0,33,33,33,33,33,33,33,33,33


In [265]:
X_tr = df_intent_tr['tweet_compound']

In [266]:
y_tr = df_intent_tr[['intent']]

In [267]:
# creating bag of words model 
cv = CountVectorizer(max_features = 1500) 
  
X = cv.fit_transform(X_tr).toarray() 
y = y_tr.iloc[:, 0].values 

# fitting naive bayes to the training set 
from sklearn.naive_bayes import MultinomialNB 

classifier = MultinomialNB();
classifier.fit(X, y) 

MultinomialNB()

In [290]:
df_intent_pred = pd.read_csv('data_training.csv')

In [291]:
df_intent_pred = df_intent_pred[df_intent_pred['intent'].isnull()]
df_intent_pred.reset_index(drop=True,inplace=True)

In [292]:
df_for_pred = df_intent_pred.sample(400)

In [294]:
X_test = cv.fit_transform(df_for_pred['tweet_compound']).toarray() 

In [295]:
y_pred = classifier.predict(X_test) 

In [302]:
df_for_pred['pred'] =  classifier.predict(X_test)

In [304]:
#df_for_pred.to_csv('preds1.csv')

### Round 2

In [378]:
df_round2 = pd.read_csv('preds1.csv')

In [379]:
df_intent_tr.reset_index(drop=True,inplace=True)

In [380]:
df_round2 = df_round2.drop(columns='pred',axis=1)
df_round2.reset_index(drop=True,inplace=True)

In [381]:
df_intent_tr2 = pd.concat([df_intent_tr,df_round2])

In [382]:
df_intent_tr2.head(2)

,Unnamed: 0,username,tweet,tweet_compound,intent,topic,neg,neu,pos,compound,Unnamed: 0.1
0,89389.0,mstothard,can it sustain its evergrowing costs and...,can it sustain its evergrowing costs and...,5.0,2.0,0.0,0.88,0.12,0.743,NaN
1,65049.0,saadishtiaq,is premium and metal account subscription bas...,is premium and metalaccount subscription base...,3.0,2.0,0.0,1.00,0.00,0.000,NaN


In [383]:
X_tr2 = df_intent_tr2['tweet_compound']

In [384]:
y_tr2 = df_intent_tr2[['intent']]

In [385]:
X_test_round2 = test.sample(400)

In [386]:
# creating bag of words model 
cv2 = CountVectorizer(max_features = 1500) 
  
X2 = cv2.fit_transform(X_tr2).toarray() 
y2 = y_tr2.iloc[:, 0].values 

# fitting naive bayes to the training set 
#from sklearn.naive_bayes import MultinomialNB 

classifier2 = MultinomialNB();
classifier2.fit(X2, y2) 

MultinomialNB()

In [387]:
X_test2 = cv2.fit_transform(X_test2['tweet_compound']).toarray() 

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [388]:
y_pred2 = classifier2.predict(X_test2) 

In [390]:
X_test_round2['pred'] =  classifier2.predict(X_test2)

In [391]:
X_test_round2

,username,tweet,tweet_compound,topic,neg,neu,pos,compound,pred
13739,cicconeraphael,revolut can i have an answer in the chat i ne...,revolut can i have an answer in the chat i ne...,4,0.000,0.729,0.271,0.6369,9.0
21675,shakhsjo,i sent even less but u guys blocked my account,i sent even less but u guys lock my account,0,0.000,1.000,0.000,0.0000,9.0
80485,manonthegreen,simple things like alerts when a currency bal...,simple things like alerts when a currency bal...,3,0.000,0.896,0.104,0.6124,5.0
40870,vianou13,thank you for your quick reaction,thank you for your quick reaction,2,0.000,0.667,0.333,0.3612,5.0
26210,johann_vanlindt,now they just need become profitable an...,now they just need become profitable an...,2,0.000,0.791,0.209,0.4404,5.0
...,...,...,...,...,...,...,...,...,...
21889,jydirl,cant topup with any debit card its prime day ...,cant topup with any debitcard its prime day help,4,0.000,0.748,0.252,0.4019,5.0
92311,bledjr,actually problem solved but yes i had to ex...,actually problem solved but yes i had to ex...,0,0.034,0.611,0.355,0.9538,5.0
8768,mschfer9,every cent you invest here at revolut is ...,every cent you invest here at revolut is ...,2,0.219,0.627,0.154,-0.3353,6.0
71676,mariselasalas,revolut sucks i want to close my account and ...,revolut sucks i want to close my account and ...,2,0.128,0.710,0.163,-0.0166,1.0
